This program collects the corpus of school books on Geometry from https://docbaza.ru.

In [1]:
import urllib.request
import re
import os
import shutil
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'

In [5]:
common_link = "https://docbaza.ru"
links = ["/urok/geometr/07/001/", "/urok/geometr/07/003/", "/urok/geometr/07/005/", "/urok/geometr/07/006/", "/urok/geometr/10/001/", "/urok/geometr/10/004/", "/urok/geometr/10/005/", "/urok/geometr/10/007/", ]

In [12]:
def download_html(pageUrl): ## downloading text of an html page
    try:
        page = urllib.request.Request(pageUrl, headers={'User-Agent':user_agent})
        with urllib.request.urlopen(page) as hpage:
            html = hpage.read().decode('windows-1251')
        return html
    except:
        print('there\'s no such page', pageUrl)
        html = None
        return html

In [ ]:
def get_numbofpages(page): ## getting number of html pages for a book
    regPages = re.compile('Страницы учебника:.*?(\d+)</A> </div></p>', re.DOTALL)
    if re.search(regPages, page):
        total = re.search(regPages, page).group(1)
    return total

In [89]:
def get_cleantext(page): ## getting clean text from the page
    regText = re.compile('текст страницы, которая находится выше\):(.*?)<hr>', re.DOTALL)
    regSub = re.compile('<sub>.*?</sub>', re.DOTALL)
    regSup = re.compile('<sup>.*?</sup>', re.DOTALL)
    regUnder = re.compile('<p><sub>.*?</sub>.*?\.</p>', re.DOTALL)
    regTag = regTag = re.compile('<.*?>', re.DOTALL)
    if re.search(regText, page):
        text = re.search(regText, page).group(1)
    text = re.sub(regSub,'', text)
    text = re.sub(regUnder,'', text)
    text = re.sub(regSup,'', text)
    text = re.sub(regTag, ' ', text)
    text = re.sub(r'&.*?;', '', text)
    text = re.sub(r'[Рр]ис\. \d+? ', '', text)
    text = re.sub(r'(\n)+', '\n', text)
    text = re.sub(r'(\s)+', ' ', text)
    return text

In [104]:
def get_books(): ## getting all geometry books from the site
    for idx, link in enumerate(links):
        main_link = common_link + link
        mainpage = download_html(main_link)
        total = get_numbofpages(mainpage)
        for i in range(5, int(total)):
            if len(str(i)) == 1:
                current = download_html(main_link + '00' + str(i) + '.html')
            elif len(str(i)) == 2:
                current = download_html(main_link + '0' + str(i) + '.html')
            else:
                current = download_html(main_link + str(i) + '.html')
            curr_text = get_cleantext(current)
            with open('geometry/book' + str(idx) + '.txt', 'a', encoding = 'utf-8') as b:
                b.write(curr_text)

In [ ]:
get_books() # got all books

In [66]:
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences

C:\Users\Sony\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
## cleaning the book which was collected manually
with open('book8.txt', 'r', encoding='utf-8') as b:
    book = b.read()

regQL = re.compile('\?[A-Za-z]+')
regLt = re.compile(' [A-Za-z]+')
regOp = re.compile('[\\=+*°<>/]')
book = re.sub(regOp, '', book)
book = re.sub(r'\d\)', '', book)
book = re.sub(r'\d\.', '', book)
book = re.sub(r'\d', '', book)
book = re.sub(regQL, '', book)
book = re.sub(regLt, '', book)
book = re.sub(r'\?{2,}', '', book)
book = re.sub(r' +', ' ', book)
book = re.sub(r'(\n|\n )+', '\n', book)

with open('new_book8.txt', 'w', encoding='utf-8') as b:
    b.write(book)

In [54]:
#got more o less clean texts for the first 8 books by changing the book titles in "with open(title, ...)"

with open('book7.txt', 'r', encoding='utf-8') as b:
    book = b.read()

regPic = re.compile('(Рис\.|рис\.)')
regBrPic = re.compile('\(.*?рис\..*?\)')
regLt = re.compile(' [A-Za-z]+')
regOp = re.compile('[\\=+*°<>/\|]')
book = re.sub(regOp, '', book)
book = re.sub(r'&.*?;', '', book)
book = re.sub(r'\d\)', '', book)
book = re.sub(r'\d\.', '', book)
book = re.sub(r'§ ', '', book)
book = re.sub(r'\d', '', book)
book = re.sub(regBrPic, '', book)
book = re.sub(regPic, '', book)
book = re.sub(r' \( ', '', book)
book = re.sub(regLt, '', book)
book = re.sub(r' +', ' ', book)
book = re.sub(r'(\n|\n )+', '\n', book)

with open('new_book7.txt', 'w', encoding='utf-8') as b:
    b.write(book)

In [ ]:
#got all books in one file
for root, dirs, files in os.walk('geometry'):
    for name in files:
        with open(os.path.join('geometry', name), 'r', encoding='utf-8') as b:
            book = b.read()
        with open('all geometry.txt', 'a', encoding = 'utf-8') as b:
            b.write(book)

with open('all geometry.txt', 'r', encoding = 'utf-8') as b:
    allbook = b.read()